Google Jobs


We use SerpAPI provided by serpapi.com to scrape features and create a dataset 

(due to rate limit of only 1 page only 10 rows was scraped)

In [7]:
import requests
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta

api_url = "https://serpapi.com/search"
query_params = {
    "engine": "google_jobs",
    "q": "Artificial Intelligence engineer",
    "location": "India",
    "api_key": "3eea8e9ee1fccbab086499d46a6218e3abc18aa4e532ad24a9beb06dea355fb1"
}


In [8]:
def scrape_article_ids(base_url, max_pages, query_params):
    job_title = []
    employer_name = []
    job_city = []
    job_country = []
    job_type = []
    job_description = []
    job_posted_at_datetime_utc = []
    job_posted = []

    try:
        response = requests.get(base_url, params=query_params)
        if response.status_code == 200:
            data = response.json()
            created_at = datetime.strptime(data["search_metadata"]["created_at"], "%Y-%m-%d %H:%M:%S %Z")
            jobs = data.get("jobs_results", [])

            def get_posted_datetime(posted_at_str, ref_time):
                posted_at_str = posted_at_str.lower()
                if "day" in posted_at_str:
                    return ref_time - timedelta(days=int(posted_at_str.split()[0]))
                elif "hour" in posted_at_str:
                    return ref_time - timedelta(hours=int(posted_at_str.split()[0]))
                elif "minute" in posted_at_str:
                    return ref_time - timedelta(minutes=int(posted_at_str.split()[0]))
                return ref_time

            for job in jobs:
                job_title.append(job.get("title", ""))
                employer_name.append(job.get("company_name", ""))
                location = job.get("location", "")
                job_city.append(location.split(",")[0] if "," in location else location)
                job_country.append("India")
                job_type.append(job.get("detected_extensions", {}).get("schedule_type", ""))
                job_description.append(job.get("description", ""))

                posted_at_raw = job.get("detected_extensions", {}).get("posted_at", "")
                job_posted.append(posted_at_raw)

                if posted_at_raw:
                    actual_time = get_posted_datetime(posted_at_raw, created_at)
                    job_posted_at_datetime_utc.append(actual_time.strftime("%Y-%m-%d %H:%M:%S"))
                else:
                    job_posted_at_datetime_utc.append("")

        else:
            print("Error:", response.status_code, response.text)
    except Exception as e:
        print("Exception occurred:", e)

    return job_title, employer_name, job_city, job_country, job_type, job_description, job_posted_at_datetime_utc, job_posted


In [9]:
max_pages = 1  #since serpapi only limits to one page
job_title, employer_name, job_city, job_country, job_type, job_description, job_posted_at_datetime_utc, job_posted = scrape_article_ids(api_url, max_pages, query_params)

In [10]:
df = pd.DataFrame({
    "job_title": job_title,
    "employer_name": employer_name,
    "job_city": job_city,
    "job_country": job_country,
    "job_type": job_type,
    "job_description": job_description,
    "job_posted": job_posted,
    "job_posted_at_datetime_utc": job_posted_at_datetime_utc
})



In [11]:
df

,job_title,employer_name,job_city,job_country,job_type,job_description,job_posted,job_posted_at_datetime_utc
0,AI Engineer,Teradata,Hyderabad,India,Full-time,AI Engineer: Shape the Future of Autonomous In...,,
1,ML Engineer II - Applied AI,Uber,Bengaluru,India,Full-time,ABOUT THE ROLE\n\nApplied AI is a horizontal A...,26 days ago,2025-07-08 13:08:53
2,Engineer/Sr. Engineer - Gen AI/ML,Qualcomm,Hyderabad,India,Full-time,Company:\nQualcomm India Private Limited\n\nJo...,,
3,Sr Engineer-AI/ML,Target India,Bengaluru,India,Full-time,About Us\n\nAs a Fortune 50 company with more ...,,
4,OpenShift AI Engineer,Pearson Carter,Delhi Division,India,Full-time,OpenShift AI Engineer\n\nPearson Carter are cu...,2 days ago,2025-08-01 13:08:53
5,Sr. Data and AI Engineer,Philips,Bengaluru,India,Full-time,Job Title\nSr. Data and AI Engineer\n\nJob Des...,5 days ago,2025-07-29 13:08:53
6,"AI Engineer, Generative AI & Agents",EVRY India,Bagaluru,India,Full-time,About Us\n\nWe are developers of digital futur...,4 days ago,2025-07-30 13:08:53
7,AI Platform Engineer,Adobe,Bengaluru,India,Full-time,Our Company\n\nChanging the world through digi...,28 days ago,2025-07-06 13:08:53
8,AI/ML Engineer - Specialist,Equifax,Pune,India,Full-time,"What You’ll Do\n• Design, develop and deploy a...",6 days ago,2025-07-28 13:08:53
9,Artificial Intelligence / Machine Learning Eng...,Meril,Karnataka,India,Full-time,Job Title: Machine Learning Engineer\n\nLocati...,19 days ago,2025-07-15 13:08:53


In [12]:
df.to_csv("serpapi_ai_jobs.csv", index=False)